In [1]:
!pip install transformers accelerate bitsandbytes sentencepiece --quiet


In [2]:
!pip install sacrebleu bert-score --quiet


In [3]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


CUDA available: False
GPU count: 0
Device name: None


/home/ethicaldevice/Desktop/Titu_exp/t2env/lib/python3.12/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Choose model
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# Load model in bfloat16 with device_map="auto" for GPU
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# Create pipeline for text generation
translator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)


/home/ethicaldevice/Desktop/Titu_exp/t2env/lib/python3.12/site-packages/torch/cuda/__init__.py:829: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


In [5]:
def make_prompt(text, src_lang="Hinglish", tgt_lang="English"):
    return f"""You are a helpful translation assistant.
Translate the following text from {src_lang} to {tgt_lang}.

Text: {text}

Translation:"""


In [6]:
import json

sample = {
    "Sentences":"New Delhi. देश (India) में कोरोना वायरस (Corona Virus) का कहर कम होने का नाम नहीं ले रहा है।",
    "Id":0
}

prompt = make_prompt(sample["Sentences"], src_lang="Code-mixed Hindi-English", tgt_lang="English")

output = translator(
    prompt,
    max_new_tokens=128,
    temperature=0.2,
    do_sample=False
)

print("Prompt:\n", prompt)
print("\nGenerated Translation:\n", output[0]["generated_text"])


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Prompt:
 You are a helpful translation assistant.
Translate the following text from Code-mixed Hindi-English to English.

Text: New Delhi. देश (India) में कोरोना वायरस (Corona Virus) का कहर कम होने का नाम नहीं ले रहा है।

Translation:

Generated Translation:
 You are a helpful translation assistant.
Translate the following text from Code-mixed Hindi-English to English.

Text: New Delhi. देश (India) में कोरोना वायरस (Corona Virus) का कहर कम होने का नाम नहीं ले रहा है।

Translation: New Delhi. The Corona Virus is not letting the country (India) recover from its effects.

Note: Code-mixed Hindi-English refers to a language that combines elements of Hindi and English, often used in informal communication, especially in India. The translation aims to preserve the original tone and meaning of the text. 

Please let me know if you need any further assistance.


In [ ]:
import os, json
import pandas as pd
from tqdm import tqdm

# Load dataset
df = pd.read_json(
    "/home/ethicaldevice/Desktop/Titu_exp/mt_comilingua_test.json",
    lines=True
)

translations = []
progress_path = "progress.json"
checkpoint_path = "llama_3.2.3b_checkpoint.json"

# Clean old logs if rerunning
if os.path.exists(progress_path):
    os.remove(progress_path)
if os.path.exists(checkpoint_path):
    os.remove(checkpoint_path)

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Translating"):
    prompt = make_prompt(row["Sentences"], src_lang="Code-mixed Hindi-English", tgt_lang="English")
    out = translator(prompt, max_new_tokens=128, temperature=0.2, do_sample=False)
    generated = out[0]["generated_text"].split("Translation:")[-1].strip()

    # Save in-memory
    translations.append(generated)

    # Append to progress.json (streaming log)
    progress_record = {
        "Id": int(row["Id"]),
        "Input": row["Sentences"],
        "llama_translation": generated
    }
    with open(progress_path, "a", encoding="utf-8") as f:
        f.write(json.dumps(progress_record, ensure_ascii=False) + "\n")

    # Every 50 rows → save checkpoint.jsonl
    if (idx + 1) % 50 == 0:
        temp_df = df.iloc[: idx + 1].copy()
        temp_df["llama_translation"] = translations
        temp_df.to_json(checkpoint_path, orient="records", lines=True, force_ascii=False)

# Final save
df["llama_translation"] = translations
df.to_json("llama_translations.json", orient="records", lines=True, force_ascii=False)

print("✅ Finished. Final → llama_translations.json")
print(f"Live log → {progress_path}")
print(f"Checkpoints → {checkpoint_path}")


Translating:   0%|          | 0/5000 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Translating:   0%|          | 1/5000 [00:15<21:37:10, 15.57s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Translating:   0%|          | 2/5000 [00:34<24:04:19, 17.34s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Translating:   0%|          | 3/5000 [00:55<26:26:48, 19.05s/it]The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMER

In [ ]:
import pandas as pd
import sacrebleu
from bert_score import score

# Load predictions + refs
df = pd.read_json("llama_translations.jsonl", lines=True)

refs = df["Annotator_1_en_translation"].astype(str).tolist()
hyps = df["llama_translation"].astype(str).tolist()

# --- BLEU ---
bleu = sacrebleu.corpus_bleu(hyps, [refs])
print(f"BLEU: {bleu.score:.2f}")

# --- BERTScore ---
P, R, F1 = score(hyps, refs, lang="en", model_type="microsoft/deberta-xlarge-mnli")
print(f"BERTScore Precision: {P.mean().item():.4f}")
print(f"BERTScore Recall:    {R.mean().item():.4f}")
print(f"BERTScore F1:        {F1.mean().item():.4f}")
